In [2]:
import mysql.connector

In [4]:
#db = mysql.connector.connect(auth_plugin="", user="", password="")

## Q1: Some of the facilities charge a fee to members, but some do not.  Please list the names of the facilities that do. 

In [ ]:
SELECT * FROM Facilities WHERE membercost > 0;

## Q2: How many facilities do not charge a fee to members?

In [ ]:
SELECT COUNT(*) FROM Facilities WHERE membercost = 0;

## Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE (membercost > 0) AND (membercost < 0.2 * monthlymaintenance);

## Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [ ]:
SELECT * FROM Facilities WHERE facid = 1 UNION SELECT * FROM Facilities WHERE facid = 5;

## Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than 100? Return the name and monthly maintenance of the facilities in question.

In [ ]:
SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance >100 THEN  'expensive' ELSE  'cheap' END AS TYPE FROM Facilities;

## Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution. 

In [ ]:
SELECT firstname, surname, MAX(joindate) FROM Members WHERE surname != 'GUEST';

## Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [ ]:
SELECT DISTINCT CONCAT(members.firstname, ' ', members.surname) AS full_name \
                FROM country_club.Members members \
                JOIN (SELECT bookings.facid, bookings.memid FROM country_club.Bookings bookings \
                JOIN country_club.Facilities facilities \
                ON bookings.facid = facilities.facid AND facilities.name LIKE '%Tennis Court%') sub \
                ON members.memid = sub.memid AND members.firstname NOT LIKE 'GUEST%' ORDER BY 1

## Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than 30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries. 

In [ ]:
SELECT facilities.name, CONCAT(members.firstname,  ' ', members.surname) as full_name,\
                CASE WHEN bookings.memid = 0 THEN facilities.guestcost * bookings.slots \
                     ELSE facilities.membercost * slots END AS cost \
                FROM country_club.Bookings bookings \
                JOIN country_club.Members members \
                ON bookings.memid = members.memid AND DATE(bookings.starttime) = '2012-09-14' \
                JOIN country_club.Facilities facilities \
                ON bookings.facid = facilities.facid HAVING cost > 30 ORDER BY 3 DESC

## Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
SELECT sub.fac_name, sub.full_name, sub.cost \
FROM (SELECT facilities.name AS fac_name, CONCAT(members.firstname,  ' ', members.surname) AS full_name, \
             CASE WHEN bookings.memid = 0 THEN facilities.guestcost * bookings.slots \
             ELSE facilities.membercost * slots END AS cost \
      FROM country_club.Bookings bookings \
      JOIN country_club.Members members ON bookings.memid = members.memid AND DATE(bookings.starttime) = '2012-09-14' \
      JOIN country_club.Facilities facilities ON bookings.facid = facilities.facid) sub \
      WHERE sub.cost > 30 \
      ORDER BY 3 DESC

## Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [ ]:
SELECT facilities.name, \
                SUM(CASE WHEN bookings.memid = 0 THEN facilities.guestcost * bookings.slots \
                         ELSE facilities.membercost * bookings.slots END) AS total_revenue \
                FROM country_club.Facilities facilities \
                JOIN country_club.Bookings bookings \
                ON facilities.facid = bookings.facid \
                GROUP BY 1 HAVING total_revenue <1000 ORDER BY 2